In [37]:
import requests

In [94]:
api_key = "RGAPI-baf2381d-4899-4219-b229-d8f22c1ffd8e"
username = "Friskis1"
MatchNumber = "5"

In [88]:
api_AccountURL = "https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + username
print(api_AccountURL)

api_AccountURLandAPI = api_AccountURL + '?api_key=' + api_key
print(api_AccountURLandAPI)

responseAccount = requests.get(api_AccountURLandAPI)
player_info = responseAccount.json()
# print(player_info)

puuid = player_info['puuid']
# print(puuid)

api_MatchHistory = "https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/" + puuid + "/ids?start=0&count=" + MatchNumber
# print(api_MatchHistory)

api_MatchHistoryAPI = api_MatchHistory + '&api_key=' + api_key 
# print(api_MatchHistoryAPI)

responseMatchHistory = requests.get(api_MatchHistoryAPI)
match_ids = responseMatchHistory.json()
print(match_ids)

https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Friskis1
https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Friskis1?api_key=RGAPI-baf2381d-4899-4219-b229-d8f22c1ffd8e
['EUW1_6463105760', 'EUW1_6463058427', 'EUW1_6463017654', 'EUW1_6462937763', 'EUW1_6462905444']


In [100]:
def calculate_combined_gold(participants):
    total_gold = 0
    for participant in participants:
        total_gold += participant['goldEarned']
    return total_gold

In [106]:
for i in range(5):
    api_MatchData = "https://europe.api.riotgames.com/lol/match/v5/matches/" + match_ids[i] + '?api_key=' + api_key
    # print(api_MatchData)

    responseMatchData = requests.get(api_MatchData)
    match_data = responseMatchData.json()

    # Calculate combined gold of the first 5 players
    team1_participants = match_data['info']['participants'][:5]
    team1_gold = calculate_combined_gold(team1_participants)
    print("Combined gold of the first 5 players:", team1_gold)

    # Calculate combined gold of the last 5 players
    team2_participants = match_data['info']['participants'][-5:]
    team2_gold = calculate_combined_gold(team2_participants)
    print("Combined gold of the last 5 players:", team2_gold)
    
    
    
    print("\n")


Combined gold of the first 5 players: 27627
Combined gold of the last 5 players: 16111


Combined gold of the first 5 players: 34884
Combined gold of the last 5 players: 21367


Combined gold of the first 5 players: 57841
Combined gold of the last 5 players: 41480


Combined gold of the first 5 players: 52777
Combined gold of the last 5 players: 65601


Combined gold of the first 5 players: 17783
Combined gold of the last 5 players: 11968


